In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce
import seaborn as sns
from scipy.stats import binned_statistic
from matplotlib.lines import Line2D

In [2]:
%cd ../..

/nfs/homedirs/fuchsgru/MastersThesis


In [12]:
import data.constants as dc

In [3]:
collection_name = 'week17_dropout_all_datasets'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    # print(ex['result'].keys())
    ex['metrics'] = ex['result']['results']

In [42]:
METRIC = 'Metric'
OOD_AUROC = 'OOD AUC-ROC'
OOD_AUCPR = 'OOD AUC-PR'
MISCLASSICIFACTION_AUROC = 'Misclassification AUC-ROC'
MISCLASSICIFACTION_AUCPR = 'Misclassification AUC-PR'

DATASET = 'Dataset'

SETTING = 'Setting'
HYBRID = 'Inductive'
TRANSDUCTIVE = 'Transductive'
LOC = 'Leave Out Classes'
NORMAL = 'Normal'
BERNOULLI = 'Bernoulli'
NO_EDGES = 'No Edges'
PROXY = 'Proxy'
EXPERIMENT = 'Experiment'

OOD_TYPE = 'OOD Type'


RESIDUAL = 'Residual'
SPECTRAL_NORM = 'Spectral Norm'
WEIGHT_SCALE = 'Weight Scale'
LOWER_LIPSCHITZ = 'Empirical Lower Lipschitz Bound'
UPPER_LIPSCHITZ = 'Empirical Upper Lipschitz Bound'



EPISTEMIC = 'Epistemic'
ALEATORIC = 'Aleatoric'

ACCURACY = 'Accuracy'
ACCURACY_ID = 'In-distribution Accuracy'
ECE = 'Expected Calibration Error'

In [38]:
set(ex['config']['model']['residual'] for ex in experiments), \
    set(ex['config']['model']['use_spectral_norm'] for ex in experiments)

({False}, {False})

In [69]:
metrics = set()
for ex in experiments:
    for m in [m for m in ex['metrics'].keys() if 'ood_auroc' in m]:
        metrics.add(m)

list(metrics)

['ood_auroc_max-score_normal_val',
 'ood_auroc_total-predictive-entropy_loc_val',
 'ood_auroc_logit-energy_ber_val',
 'ood_auroc_total-predictive-entropy_loc_test',
 'ood_auroc_logit-energy_ber_test',
 'ood_auroc_total-predictive-entropy_loc-no-edges_val',
 'ood_auroc_max-score_normal_test',
 'ood_auroc_max-score_loc-no-edges_test',
 'ood_auroc_total-predictive-entropy_ber-no-edges_val',
 'ood_auroc_logit-energy_normal_val',
 'ood_auroc_max-score_ber-no-edges_test',
 'ood_auroc_max-score_ber-no-edges_val',
 'ood_auroc_total-predictive-entropy_normal_test',
 'ood_auroc_logit-energy_loc-no-edges_val',
 'ood_auroc_logit-energy_ber-no-edges_val',
 'ood_auroc_max-score_normal-no-edges_val',
 'ood_auroc_logit-energy_loc-no-edges_test',
 'ood_auroc_total-predictive-entropy_ber-no-edges_test',
 'ood_auroc_logit-energy_normal-no-edges_test',
 'ood_auroc_max-score_loc-no-edges_val',
 'ood_auroc_max-score_ber_val',
 'ood_auroc_logit-energy_loc_test',
 'ood_auroc_max-score_loc_val',
 'ood_auroc_ma

In [40]:
mode = 'test'

In [47]:
data_proxy, data_acc_ece = [], []
for ex in experiments:
    cfg = ex['config']
    base = {
        SETTING : {dc.HYBRID : HYBRID, dc.TRANSDUCTIVE : TRANSDUCTIVE}[cfg['data']['setting']],
        DATASET : cfg['data']['dataset'],
    }
    data_acc_ece.append(base | {
        OOD_TYPE : cfg['data']['ood_type'],
        ACCURACY : ex['metrics'][f'accuracy_{mode}__{mode}'][0],
        ECE : ex['metrics'][f'ece_{mode}__{mode}'][0]['value'],
    })
    if cfg['data']['ood_type'] == dc.PERTURBATION:
        ood_types = (
            (BERNOULLI, 'ber'),
            (NORMAL, 'normal'),
        )
    elif cfg['data']['ood_type'] == dc.LEFT_OUT_CLASSES:
        ood_types = (
            (LOC, 'loc'),
        )
    else:
        raise ValueError(cfg['data']['ood_type'])
    
    for ood_type, ood_name in ood_types:
        
        data_proxy += [
            base | {
                EXPERIMENT : ood_type,
                PROXY : ALEATORIC,
                
            }
        ]
        
        
        metrics = {
            EXPERIMENT : ood_type,
        }
        
        
    
df_proxy = pd.DataFrame(data_proxy)
df_acc_ece = pd.DataFrame(data_acc_ece)

In [58]:
df_acc_ece[(df_acc_ece[OOD_TYPE] == 'left-out-classes') & (df_acc_ece[DATASET] == dc.PUBMED)]

,Setting,Dataset,OOD Type,Accuracy,Expected Calibration Error
600,Transductive,pubmed,left-out-classes,NaN,0.048003
601,Transductive,pubmed,left-out-classes,NaN,0.086136
602,Transductive,pubmed,left-out-classes,NaN,0.034601
603,Transductive,pubmed,left-out-classes,NaN,0.034265
604,Transductive,pubmed,left-out-classes,NaN,0.057586
...,...,...,...,...,...
695,Inductive,pubmed,left-out-classes,NaN,0.097697
696,Inductive,pubmed,left-out-classes,NaN,0.083525
697,Inductive,pubmed,left-out-classes,NaN,0.096602
698,Inductive,pubmed,left-out-classes,NaN,0.074701
